<a href="https://colab.research.google.com/github/kkmax1015/Kaggle/blob/main/Titanic/1_SampleNoteBook_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#目標：ロジスティック回帰を使用してとりあえず予測結果を出してみる
まずはロジスティク回帰を使用して、予測結果の算出までの流れを体験してみる。

###1.データを読み込み、どんな項目があるか見てみる

In [1]:
# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/Titanic/'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Titanic
1_SampleNoteBook_LogisticRegression  data  output


In [2]:
# PandasをImportする
import pandas as pd

In [3]:
# CSVデータをPandasデータフレームへ読み込む
train = pd.read_csv("./data/train.csv");
test = pd.read_csv("./data/test.csv");
submit = pd.read_csv("./data/gender_submission.csv");

In [4]:
# トレーニング用データの先頭5行を表示
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# テスト用データの先頭5行を表示
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
# 提出ファイルのサンプルを確認
submit.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [7]:
# それぞれのデータの件数を算出する
print(len(train), len(test),len(submit))

891 418 418


トレーニングデータとテストデータの見比べると、テストデータには「Survived」の項目がないことがわかる。これは、「Survived」は予測するべき答え(0：死亡 1：生存)である為。
トレーニング用データには回答が用意されている為、このトレーニングデータを使用して機械学習モデルを作成していく。

データの各項目の意味はKaggleに説明がある為、そちらを参照。

https://www.kaggle.com/c/titanic/data

###2.ロジスティック回帰モデルを作成してみる

scikit-learnのロジスティック回帰 (logistic regression) モデルを使用して、機械学習モデルを作成してみる。

scikit-learn：Pythonのオープンソース機械学習ライブラリ。主要なライブラリは実装されているので、深層学習(ニューラルネットワーク)以外は大体このライブラリで何とかなる。

ロジスティック回帰：回帰という名称がついているが、分類問題で使用される。次の記事を参照。
https://qiita.com/katsu1110/items/e4ef613559f02f183af5

scikit-learnのロジスティック回帰のページは以下。使い方でわからないものがあればまずは下記のページを参照すること。

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [8]:
# sklearnのロジスティック回帰をImport
from sklearn.linear_model import LogisticRegression

In [9]:
# ロジスティック回帰モデルの作成(初期化)
# random_state：初期化パラメータ。指定することで学習結果を固定することが出来る。
clf = LogisticRegression(random_state=0)

In [10]:
# トレーニングデータから答えを分割
y_train = train['Survived']
X_train = train.drop('Survived', axis = 1)

In [11]:
# 学習
clf.fit(X_train,y_train)

ValueError: ignored

直接入れようとしても「Stringは扱えないぞ」と怒られる。機械学習で特徴量(説明変数)として使用する為には、数値へ変換する必要がある。

In [12]:
# ひとまず数値じゃない特徴量を全て削除してしまう
X_train2 = X_train.copy()
X_train2.drop(['Name','Sex','Ticket','Cabin','Embarked'],axis = 1,inplace=True)
X_train2.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,1,3,22.0,1,0,7.2500
1,2,1,38.0,1,0,71.2833
2,3,3,26.0,0,0,7.9250
3,4,1,35.0,1,0,53.1000
4,5,3,35.0,0,0,8.0500


In [13]:
# 再度学習
clf.fit(X_train2,y_train)

ValueError: ignored

今度は異なるエラーが出ている。「NaN」があると怒られているので、おそらく値にNullが混じった項目がある。

In [14]:
# 項目別にNullの数を確認してみる
X_train2.isnull().sum()

PassengerId      0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

項目別に確認すると、「Age」にNullが177行存在することが分かる。

In [15]:
# Ageの削除
X_train2.drop('Age',axis = 1,inplace=True)

In [16]:
# 再度学習
clf.fit(X_train2,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Nullの項目であるAgeを削除したことで、学習をすることが出来た。

### 3.提出ファイルの作成

ロジスティック回帰モデルを作成したので、実際にテストデータの予測をしてみる。

In [17]:
# トレーニングデータと同じ特徴量にする
X_test = test.drop(['Name','Sex','Ticket','Cabin','Embarked','Age'],axis = 1)

# 予測値の算出
y_pred = clf.predict(X_test)

ValueError: ignored

トレーニングデータと同じ項目を削除してはずだが、エラーが発生してしまう。

In [18]:
# 項目別にNullの数を確認してみる
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

テストデータを確認すると、トレーニングデータでは大丈夫であった「Fare」にNullとなっている列が1行存在することが分かる。

トレーニングとテストのデータの状態・傾向は必ずしも同一ではない。

In [19]:
# トレーニングとテストの特徴量は同じにしないといけない為、双方から削除する。
X_train2.drop('Fare',axis = 1,inplace=True)
X_test.drop('Fare',axis = 1,inplace=True)

In [20]:
# 特徴量が変わった為、再度モデルを学習させる
clf.fit(X_train2,y_train)

# 予測値の算出
y_pred = clf.predict(X_test)

In [21]:
# 予測値の確認
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,

In [22]:
# 提出ファイルの作成
sub = submit
sub['Survived'] = y_pred
sub.to_csv('./output/submission.csv', index=False)